In [ ]:
#Importing libraries required
import numpy as np
import pandas as pd
from google.colab import files
import sys
sys.setrecursionlimit(40000)  #changing recursion limit

url="https://raw.githubusercontent.com/Naman-Ladha/ADA/main/salary%20colab.csv"
dataset= pd.read_csv (url )
print(dataset.head) #verifying the dataset
target_dataset="Salary" #Setting the target Variable
print("Shape ",dataset.shape) #printing the shape of the dataset
mode_dataset=np.unique(dataset[target_dataset])[np.argmax(np.unique(dataset[target_dataset],return_counts=True)[1])]
print("mode of the data set is", mode_dataset)

<bound method NDFrame.head of         Employment class Education level  ...      Sex  Salary
0                Private         HS-grad  ...   Female   <=50K
1                Private         HS-grad  ...     Male   <=50K
2                Private         HS-grad  ...   Female   <=50K
3                Private    Some-college  ...     Male    >50K
4           Self-emp-inc     Prof-school  ...     Male    >50K
...                  ...             ...  ...      ...     ...
35144            Private         HS-grad  ...     Male   <=50K
35145            Private         Masters  ...     Male    >50K
35146   Self-emp-not-inc       Bachelors  ...     Male   <=50K
35147            Private            12th  ...   Female   <=50K
35148          Local-gov      Assoc-acdm  ...   Female    >50K

[35149 rows x 8 columns]>
Shape  (35149, 8)
mode of the data set is  <=50K


In [ ]:
#Computing Entropy based on the target variable
def compute_entropy(target_column):    #Function by the name of entropy which takes the coulmn values of target coulmn of the dataset
    
  if len(target_column)<2:  #Checking whether there is more than one data entry, else the entropy will be 0
    return 0
  elements, counts = np.unique(target_column,return_counts=True )  #Calculates the number of values for all the distinct values in target coulmn, and normalizes the values
  entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))]) #Entropy calculation
  return entropy
  
print(compute_entropy(dataset[target_dataset])) #Verifying the function


0.8060397101865737


In [ ]:
#Computing infomration gain from a  feature variable based on the dataset available and target values

def compute_Info_Gain(data,split_attribute_name,target_name): #Function to build information gain takes three variables- data, attribue on which data can be splitted and the target variable
   
  total_entropy = compute_entropy (data[target_name]) #Entropy of the current data set based on the target value

  values= data[split_attribute_name].value_counts(normalize= True)  #Storing the normalized occurance of the unique values of the feature against which entropy to be calculated
  
  split_ent=0
  for v, fr in values.iteritems():
    sub_ent= compute_entropy(data[data[split_attribute_name]==v][target_name]) #Calculating entropy of each split possible(unique values of attribute) based on target variable
    split_ent+=fr*sub_ent  #Adding entropy of all the data generated from the unique values of splitting attribute,  weighted with their frequency                

  Information_Gain = total_entropy - split_ent #information gain = Current entropy- (sum of weighted entropy of the dataset after splitted by split attribute)
  return Information_Gain




In [ ]:
#Building the decion tree, it takes 5 attributes, data, target value, minimum samples is the min no data for which classification can be made
#max_depth is the depth of the tree, it is used to prevent overfitting
def ID3(data,target_attribute_name,parent_node_class = None,min_num_samples=1, max_depth=10):  

    features = [i for i in data.columns[:] if i != target_attribute_name] #creating features by removing the target column or checking whether there are any features by target name
    #print(features)

    if len(np.unique(data[target_attribute_name])) <= 1: #Verifying if there is only ne unique value in the target column

        return np.unique(data[target_attribute_name])[0] #If there is only one unique target value then return that unique value
    
    elif len(data)< min_num_samples: #if the length of the samples is less than the required minimum number of samples
        return np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])] #returning the mode of the target value

    elif max_depth==0: #Verifying if the depth has been achieved, this will be achieved by recursively reducing the max depth while calling the function
            return np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])] #returning the mode of the target value

    elif len(features) ==0: #If length of the features is less than 1 we will return the current parent node class caculated
        return parent_node_class
        
    else:
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])] #mode of the current target value
        item_values = [compute_Info_Gain (data,feature,target_attribute_name) for feature in features]    #Return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values) #Returns the index of maximum value in the list of information gain of all the values in features
        best_feature = features[best_feature_index] #Selecting the best feature from the list of feature
        tree = {best_feature:{}} #adding the new feature selected to the tree dictionary
        features = [i for i in features if i != best_feature] #Making new features by dropping the next feature selected for building the subtree
        for value in np.unique(data[best_feature]): #Making child nodes for the best feature
            value = value 
            sub_data = data.where(data[best_feature] == value).dropna()    #Creating the subdata based on the each unique value of the splitting attribute
            subtree = ID3(sub_data,target_attribute_name,parent_node_class,min_num_samples ,max_depth-1 ) #Initializing a new subtree, this will make recursion, reducing the depth by 1 as a new edge is made in the tree         
            tree[best_feature][value] = subtree             #Mapping the new subtree made to the parent node
        return(tree)    


In [ ]:
#Creating a function for prediting test values from a definned decision tree classifier
"""We are making a predictor for a single data point and then calling it iteratively for the complete dataset and storing it"""
"""The predictor takes two values which is the data to be predicted and the decision tree"""
def predict_single_data(query,tree): 

    for key in list(query.keys()):  #iterating all the features to find the current splitting features based on the data
        if key in list(tree.keys()):   #finding the current splitting node in the tree

            try:
                result = tree[key][query[key]]  # tree[key] is the splitting node and tree[key][query[key]] is the value of the 
            except:
                return mode_dataset

            result = tree[key][query[key]] #value stored in the tree based on the current node
            if isinstance(result,dict): #Identifying whether we have reached the leaf node or there is more tree 
                return predict_single_data(query,result) #If the leaf node not found then recursively call the predict function based on the next splitting node
            else:
                return result


def predict(test_data, tree_test):  #Predicting the complete data 
 
  pq=test_data.iloc[:,:].to_dict(orient="records")  #Converting the data into dictionary as the tree is stored as dictionary
  prediction= [predict_single_data (pq[i],tree_test) for i in range(len(test_data))] #Calling the single data points, by sending each data point and the tree
  return prediction  #returning the list of values predicted
  


In [ ]:
#Splitting the dataset based on the percentage on which the user enters
def train_test_split(data,target, percent):
  withouttarget = [i for i in data.columns[:] if i != target] #Finding all the features except target
  records= round(len(data)*percent/100)   #Number of records based on the percentage user entered
  
  training_data_x=data[withouttarget].iloc[:records].reset_index(drop=True) #training data 
  training_data_y=data[target].iloc[:records].reset_index(drop=True)   #output of training data
  testing_data_x=data[withouttarget].iloc[records:].reset_index(drop=True) #testing data
  testing_data_y=data[target].iloc[records:].reset_index(drop=True) #output of testing data
  return training_data_x, training_data_y, testing_data_x, testing_data_y

x_train, y_train, x_test, y_test=train_test_split(dataset,target_dataset,80 ) #Splitting the dataset by 80%
x_train[target_dataset]=y_train  #Adding the target column of the training dataset


tree=ID3(x_train,target_dataset,None,40,4) #Creating the tree based on the training dataset, with minimum number of 40 data for prediction and depth of 4 
print(tree)
prediction= predict(x_test, tree)
import pprint
pprint.pprint(tree) #Printing the tree



{'Relationship status': {' Husband': {'Education level': {' 10th': {'Occupation': {' Adm-clerical': ' <=50K', ' Craft-repair': {'Employment class': {' Federal-gov': ' <=50K', ' Local-gov': ' <=50K', ' Private': ' <=50K', ' Self-emp-not-inc': ' <=50K'}}, ' Exec-managerial': ' <=50K', ' Farming-fishing': ' <=50K', ' Handlers-cleaners': ' <=50K', ' Machine-op-inspct': {'Employment class': {' Private': ' <=50K', ' Self-emp-not-inc': ' <=50K'}}, ' Other-service': ' <=50K', ' Prof-specialty': ' >50K', ' Protective-serv': ' <=50K', ' Sales': ' <=50K', ' Tech-support': ' >50K', ' Transport-moving': {'Employment class': {' Local-gov': ' <=50K', ' Private': ' <=50K', ' Self-emp-not-inc': ' <=50K', ' State-gov': ' >50K'}}}}, ' 11th': {'Occupation': {' Adm-clerical': ' <=50K', ' Craft-repair': {'Employment class': {' Local-gov': ' >50K', ' Private': ' <=50K', ' Self-emp-inc': ' >50K', ' Self-emp-not-inc': ' <=50K', ' State-gov': ' <=50K'}}, ' Exec-managerial': ' <=50K', ' Farming-fishing': ' <=50K

In [ ]:
# Determining the performance of the tree build

def performance(positive_class, actual_value,_predicted_value):
  positive=positive_class #positive class of the target 
  if(len(actual_value)==0):
    return 0
  if(len(actual_value)!=len(_predicted_value)):
    return 0
  else:
    fp=0  #false positive
    tp=0  #true positive
    tn=0  #true negative
    fn=0  #false negative

    for i in range(len(actual_value)):
      if (actual_value[i]==positive):
        if (actual_value[i]==_predicted_value[i]):
          tp=tp+1
        else:
          fn=fn+1
      else:
        if (actual_value[i]==_predicted_value[i]):
          tn=tp+1
        else:
          fp=fn+1
    
    accuracy=(tp+tn)/(tp+tn+fp+fn)
    sensitivity_true_positive_rate= tp/(tp+fn)
    specificity_true_negative_rate= tn/(tn+fp)

    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    adjusted_accuracy=(sensitivity_true_positive_rate+specificity_true_negative_rate)/2
    from termcolor import colored
    print(colored("         Performance",'blue', attrs=['bold']))
    print("____________________________________")
    print(colored("             Confusion Matrix",'green'))
    

    print("              Predicted Class")
    print("Actual Class")
    print("                 P  |   N")
    print("             _____________")
    print("             P| ",tp," |  ",fn)
    print("             N| ",fp," |  ",tn)

    print("____________________________________")
    print("Accuracy     |", round(accuracy,2))
    print("Sensitivity  |", round(sensitivity_true_positive_rate,2))
    print("Specificity  |", round(specificity_true_negative_rate),2)
    print("Precision    |", round(precision,2))
    print("Recall       |", round(recall,2))
    print("Adj_accuracy |", round(adjusted_accuracy,2))
    print("____________________________________")
    
performance(" >50K",y_test, prediction) #evaluating the performance on the testing output and predicted value and >50K is the positive class

         Performance
____________________________________
             Confusion Matrix
              Predicted Class
Actual Class
                 P  |   N
             _____________
             P|  1078  |   698
             N|  698  |   1079
____________________________________
Accuracy     | 0.61
Sensitivity  | 0.61
Specificity  | 1 2
Precision    | 0.61
Recall       | 0.61
Adj_accuracy | 0.61
____________________________________


In [ ]:
#Print tree from dictionary
def printTree(tree, d = 0):
  if (tree == None or len(tree) == 0):

    print ("\t" * d, "-") #spacing the node based on the depth of the node
  else:
    for key, val in tree.items():
      if (isinstance(val, dict)): #checking whether leaf node has been reached
        print ("\t" * d, key)
        printTree(val, d+1) #Recursively calling the tree
      else:
        print ("\t" * d, key, str('(') + val + str(')'))

printTree(tree)

 Relationship status
	  Husband
		 Education level
			  10th
				 Occupation
					  Adm-clerical ( <=50K)
					  Craft-repair
						 Employment class
							  Federal-gov ( <=50K)
							  Local-gov ( <=50K)
							  Private ( <=50K)
							  Self-emp-not-inc ( <=50K)
					  Exec-managerial ( <=50K)
					  Farming-fishing ( <=50K)
					  Handlers-cleaners ( <=50K)
					  Machine-op-inspct
						 Employment class
							  Private ( <=50K)
							  Self-emp-not-inc ( <=50K)
					  Other-service ( <=50K)
					  Prof-specialty ( >50K)
					  Protective-serv ( <=50K)
					  Sales ( <=50K)
					  Tech-support ( >50K)
					  Transport-moving
						 Employment class
							  Local-gov ( <=50K)
							  Private ( <=50K)
							  Self-emp-not-inc ( <=50K)
							  State-gov ( >50K)
			  11th
				 Occupation
					  Adm-clerical ( <=50K)
					  Craft-repair
						 Employment class
							  Local-gov ( >50K)
							  Private ( <=50K)
							  Self-emp-inc ( >50K)
							  Self-emp-not-inc ( <=50K)
